In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
import seaborn as sns
sns.set_style("darkgrid")

###################################
# User parameters
###################################
save_folder = "big-run" 
SEEDFS = [42, 43, 44]
###################################

def smooth(data, window=4):
    """Apply a simple moving average smoothing."""
    if window > 1 and len(data) >= window:
        return np.convolve(data, np.ones(window)/window, mode='valid')
    return data

# Identify all scenario folders inside the save_folder
if not os.path.exists(save_folder):
    raise FileNotFoundError(f"The folder '{save_folder}' does not exist.")

scenario_folders = [
    f for f in os.listdir(save_folder) 
    if os.path.isdir(os.path.join(save_folder, f))
]

for scenario in scenario_folders:
    scenario_path = os.path.join(save_folder, scenario)
    numpy_path = os.path.join(scenario_path, "numpy")

    # Check if the numpy directory exists and contains the required files
    required_files = [
        'all_myopic_runs.npy',
        'all_proactive_runs.npy',
        'all_reactive_runs.npy',
        'all_myopic_steps_runs.npy',
        'all_proactive_steps_runs.npy',
        'all_reactive_steps_runs.npy'
    ]
    if not os.path.exists(numpy_path):
        # Not a scenario folder with numpy results
        continue
    if not all(os.path.isfile(os.path.join(numpy_path, f)) for f in required_files):
        # Missing some required files
        continue

    # Load arrays
    all_myopic_runs = np.load(os.path.join(numpy_path, 'all_myopic_runs.npy'), allow_pickle=True)
    all_proactive_runs = np.load(os.path.join(numpy_path, 'all_proactive_runs.npy'), allow_pickle=True)
    all_reactive_runs = np.load(os.path.join(numpy_path, 'all_reactive_runs.npy'), allow_pickle=True)

    all_myopic_steps_runs = np.load(os.path.join(numpy_path, 'all_myopic_steps_runs.npy'), allow_pickle=True)
    all_proactive_steps_runs = np.load(os.path.join(numpy_path, 'all_proactive_steps_runs.npy'), allow_pickle=True)
    all_reactive_steps_runs = np.load(os.path.join(numpy_path, 'all_reactive_steps_runs.npy'), allow_pickle=True)

    # Compute mean and std
    myopic_mean = all_myopic_runs.mean(axis=0)
    myopic_std = all_myopic_runs.std(axis=0)
    proactive_mean = all_proactive_runs.mean(axis=0)
    proactive_std = all_proactive_runs.std(axis=0)
    reactive_mean = all_reactive_runs.mean(axis=0)
    reactive_std = all_reactive_runs.std(axis=0)

    myopic_steps_mean = all_myopic_steps_runs.mean(axis=0).astype(int)
    proactive_steps_mean = all_proactive_steps_runs.mean(axis=0).astype(int)
    reactive_steps_mean = all_reactive_steps_runs.mean(axis=0).astype(int)

    # Print some info
    num_seeds_myopic = all_myopic_runs.shape[0]
    num_episodes_myopic = all_myopic_runs.shape[1] if num_seeds_myopic > 0 else 0
    num_seeds_proactive = all_proactive_runs.shape[0]
    num_episodes_proactive = all_proactive_runs.shape[1] if num_seeds_proactive > 0 else 0
    num_seeds_reactive = all_reactive_runs.shape[0]
    num_episodes_reactive = all_reactive_runs.shape[1] if num_seeds_reactive > 0 else 0

    for i in range(num_seeds_myopic):
        total_steps_myopic = all_myopic_steps_runs[i].sum()
        print(f"For scenario {scenario}, seed index {i}, Myopic: finished {num_episodes_myopic} episodes, performed {total_steps_myopic} timesteps.")

    for i in range(num_seeds_proactive):
        total_steps_proactive = all_proactive_steps_runs[i].sum()
        print(f"For scenario {scenario}, seed index {i}, Proactive: finished {num_episodes_proactive} episodes, performed {total_steps_proactive} timesteps.")

    for i in range(num_seeds_reactive):
        total_steps_reactive = all_reactive_steps_runs[i].sum()
        print(f"For scenario {scenario}, seed index {i}, Reactive: finished {num_episodes_reactive} episodes, performed {total_steps_reactive} timesteps.")

    # Apply smoothing
    smooth_window = 1
    myopic_mean_sm = smooth(myopic_mean, smooth_window)
    myopic_std_sm = smooth(myopic_std, smooth_window)
    myopic_steps_sm = myopic_steps_mean[:len(myopic_mean_sm)]

    proactive_mean_sm = smooth(proactive_mean, smooth_window)
    proactive_std_sm = smooth(proactive_std, smooth_window)
    proactive_steps_sm = proactive_steps_mean[:len(proactive_mean_sm)]

    reactive_mean_sm = smooth(reactive_mean, smooth_window)
    reactive_std_sm = smooth(reactive_std, smooth_window)
    reactive_steps_sm = reactive_steps_mean[:len(reactive_mean_sm)]

    # Plotting the results
    plt.figure(figsize=(12,6))
    plt.plot(myopic_steps_sm, myopic_mean_sm, label="Myopic", color='blue')
    plt.fill_between(myopic_steps_sm, 
                     myopic_mean_sm - myopic_std_sm, 
                     myopic_mean_sm + myopic_std_sm, 
                     alpha=0.2, color='blue')

    plt.plot(proactive_steps_sm, proactive_mean_sm, label="Proactive", color='orange')
    plt.fill_between(proactive_steps_sm, 
                     proactive_mean_sm - proactive_std_sm, 
                     proactive_mean_sm + proactive_std_sm, 
                     alpha=0.2, color='orange')

    plt.plot(reactive_steps_sm, reactive_mean_sm, label="Reactive", color='green')
    plt.fill_between(reactive_steps_sm, 
                     reactive_mean_sm - reactive_std_sm, 
                     reactive_mean_sm + reactive_std_sm, 
                     alpha=0.2, color='green')

    plt.xlabel("Environment Steps (Frames)")
    plt.ylabel("Episode Reward")
    plt.title(f"Averaged Episode Rewards over {len(SEEDS)} Seeds (DQN, {scenario})")
    plt.legend(frameon=False)
    plt.grid(True)

    # plots_dir = os.path.join(scenario_path, "plots")
    # os.makedirs(plots_dir, exist_ok=True)

    # Get current datetime and extract last 4 digits of seconds
    current_time = datetime.now()
    seconds_str = str(int(current_time.timestamp()))[-4:]
    # Save the plot
    # plot_file = os.path.join(plots_dir, f"averaged_rewards_over_steps_{scenario}_{seconds_str}.png")
    # plt.savefig(plot_file)
    plt.show()

    # print(f"Averaged reward vs steps plot saved to {plot_file}\n")
